## Generation of light-curves for training NNs

The scripts is used to generate data sets containing light curves for training neural networks. 

Rewritten by: Soumya Shreeram <br>
Script adapted from: Eric Paic <br>
Date: 02nd March 2020

In [4]:
import numpy as np
import os,sys
import pickle as pkl
from astropy.io import fits
import glob

### 1. Paths to directories and Input parameters

In [14]:
current_dir = os.getcwd()
root_dir = os.path.abspath(os.path.join(current_dir, os.pardir))
data_dir = os.path.join(root_dir, "Desktop", "Year1 sem 2", "TP4b")
os.path.isdir(data_dir)
data_dir

'C:\\Users\\1199\\Desktop\\Year1 sem 2\\TP4b'

In [ ]:
datadir = "~/Desktop/"
scriptdir = "/home/epaic/Documents/Astro/TPIVb/script/"
resultdir = "/home/epaic/Documents/Astro/TPIVb/results/"
mapdir = "Q0158/FML0.9M0.01/"
storagedir = "/run/media/epaic/TOSHIBA EXT/maps/Q0158/"

# Constants used to convert pixels to physical length
einstein_r_1131= 2.5e16 #Einstein ring of RXJ1131 in cm assuming <M> = 0.3 M_0
einstein_r_03 = 3.414e16 #Einstein ring of QJ0158 in cm assuming <M> = 0.3 M_0
einstein_r_01 = einstein_r_03/np.sqrt(3) #Einstein ring of QJ0158 in cm assuming <M> = 0.1 M_0
einstein_r_001 = einstein_r_03/np.sqrt(30) #Einstein ring of QJ0158 in cm assuming <M> = 0.01 M_0
